Exercicio 2 PCS5024, Mais algoritmos para a base adult kaggle

In [ ]:
import FuncoesAuxiliares as fa
import FuncoesDeMachineLearning as fml
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
datasetPath = f"Kaggle\\adult.csv"
datasetDataFrame = fa.getDataframeInSpecificFormat(datasetPath)

Descricao das colunas discretas da base 

In [ ]:
describedDataFrame = pd.DataFrame(columns = None)
for column in datasetDataFrame.columns:
    if (datasetDataFrame[column].dtype == 'object'):
        describedDataFrame[column] = datasetDataFrame[column].describe(include=['category'])
describedDataFrame

In [ ]:
datasetDataFrame['workclass'].value_counts()

Transforma classes binarias em variaveis binarias

In [ ]:
outputName = 'income'
datasetDataFrameReducedColumns = datasetDataFrame.copy() 
datasetDataFrameReducedColumns[outputName] =  datasetDataFrame["income"].map({"<=50K":1, ">50K":0})
datasetDataFrameReducedColumns['sex'] =  datasetDataFrame["sex"].map({"Male":1, "Female":0})
# Coluna casado ou não
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].replace(['Never-married','Divorced','Separated','Widowed'], 'Single')
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'],'Married')
datasetDataFrameReducedColumns["marital.status"] = datasetDataFrameReducedColumns["marital.status"].map({"Married":1, "Single":0})
# dropa a coluna de education como int
datasetDataFrameReducedColumns.drop(['education.num'],axis = 1,inplace=True)

Porcentagem de nulos em cada uma das colunas

In [ ]:
dicOfMissingValues = {}
sizeOfData = len(datasetDataFrameReducedColumns.index)
for column in datasetDataFrameReducedColumns.columns:
    dicOfMissingValues[column] = list(datasetDataFrame[column]).count("?")*100/sizeOfData
print("Porcentagem de '?' em cada coluna ")
for key,val in  dicOfMissingValues.items(): 
    print('{key} - {val:.2f}%'.format(key = key, val = val))

Remove as linhas que possuem valores nulos

In [ ]:
datasetDataFrameWithoutColumnsWithMissingValues = fa.removeLinesWithMissingValues(datasetDataFrameReducedColumns)
print(f"Numero total de linhas = {len(datasetDataFrameReducedColumns.index)}")
print(f"Numero total de linhas que não possuem valores nulos = {len(datasetDataFrameWithoutColumnsWithMissingValues.index)}")
print("Porcentagem de linhas perdidas = ")
print(f"{(1 - len(datasetDataFrameWithoutColumnsWithMissingValues.index)/len(datasetDataFrameReducedColumns.index))*100:.2f}%")

In [ ]:
describedDataFrame = pd.DataFrame(columns = None)
for column in datasetDataFrameWithoutColumnsWithMissingValues.columns:
    if (datasetDataFrameWithoutColumnsWithMissingValues[column].dtype != 'object'):
        describedDataFrame[column] = datasetDataFrameWithoutColumnsWithMissingValues[column].describe(include=['category'])
describedDataFrame

In [ ]:
datasetDataFrameWithoutColumnsWithMissingValues['relationship'].value_counts()

Mostra imagem de correlação

In [ ]:
dummiesDataFrame = pd.get_dummies(datasetDataFrameWithoutColumnsWithMissingValues)
cor = dummiesDataFrame.corr()
cor_target = abs(cor[outputName]).sort_values(ascending=False)[1:]
cor_target

In [11]:
numberOfRelevantVariables = 5
columnsNamesToBeRemoved = list(cor_target.keys()[numberOfRelevantVariables:])
correlatedDataFrame = dummiesDataFrame.drop(columns = columnsNamesToBeRemoved)
plt.figure(figsize=(50,40))
cor = correlatedDataFrame.corr()
sns.set(font_scale=4)
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds, annot_kws={"size": 50})

NameError: name 'cor_target' is not defined

Create training validaditon and test sets

In [ ]:
numberOfRelevantVariables = 30
X_train,X_crossVal, X_test,y_train,y_crossVal, y_test = fa.prepareDatasetforTraining(datasetDataFrameWithoutColumnsWithMissingValues,outputName,numberOfRelevantVariables)

Cross Validating algorithms

In [ ]:
#KNN algorithm
knnPerformanceDataFrame = fml.applyKNNWithDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
knnPerformanceDataFrame.to_excel('knnPeformance.xlsx')
knnPerformanceDataFrame

In [ ]:
#Naieve bayes
naiveBayesPerfomanceDataFrame = fml.applyNaiveBayesWithDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
naiveBayesPerfomanceDataFrame.to_excel('NBPeformance.xlsx')
naiveBayesPerfomanceDataFrame

In [ ]:
#Logistic Regression
logisticRegressionPerformanceDataFrame = fml.applyLogisticRegressionDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
logisticRegressionPerformanceDataFrame.to_excel('LRPeformance.xlsx')
logisticRegressionPerformanceDataFrame

In [ ]:
#Decision Tree
decisionTreePerformanceDataFrame = fml.applyDecisionTreeDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
decisionTreePerformanceDataFrame.to_excel('DTPeformance.xlsx')
decisionTreePerformanceDataFrame

In [ ]:
#Random forest
randomForestPerformanceDataFrame = fml.applyRandomForestDifferentHyperparameters(X_train, X_crossVal, y_train, y_crossVal,numberOfRelevantVariables)
randomForestPerformanceDataFrame.to_excel('RFPeformance.xlsx')
randomForestPerformanceDataFrame

Treinamento final:

In [ ]:
X_new_train = np.concatenate((X_train, X_crossVal), axis=0)
y_new_train = np.concatenate((y_train, y_crossVal), axis=0)

In [14]:
numberOfRelevantVariables = 30
X_new_train,y_new_train,X_test,y_test = fa.prepareDatasetforTrainingComparationWithKaggle(datasetDataFrameWithoutColumnsWithMissingValues,outputName,numberOfRelevantVariables)
randomForestPerformanceDataFrame = fml.applyRandomForestDifferentHyperparameters(X_new_train, X_test, y_new_train, y_test,numberOfRelevantVariables)
randomForestPerformanceDataFrame

NameError: name 'datasetDataFrameWithoutColumnsWithMissingValues' is not defined